In [3]:
import pandas as pd

In [4]:
movies = pd.read_csv("data/tmdb_5000_movies.csv")
movies.describe()

,budget,id,popularity,revenue,runtime,vote_average,vote_count
count,4.803000e+03,4803.000000,4803.000000,4.803000e+03,4801.000000,4803.000000,4803.000000
mean,2.904504e+07,57165.484281,21.492301,8.226064e+07,106.875859,6.092172,690.217989
std,4.072239e+07,88694.614033,31.816650,1.628571e+08,22.611935,1.194612,1234.585891
min,0.000000e+00,5.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000
25%,7.900000e+05,9014.500000,4.668070,0.000000e+00,94.000000,5.600000,54.000000
50%,1.500000e+07,14629.000000,12.921594,1.917000e+07,103.000000,6.200000,235.000000
75%,4.000000e+07,58610.500000,28.313505,9.291719e+07,118.000000,6.800000,737.000000
max,3.800000e+08,459488.000000,875.581305,2.787965e+09,338.000000,10.000000,13752.000000


In [5]:
import ast 
movies['geners_name'] = movies['genres'].apply( 
    lambda x: [d['name'] for d in ast.literal_eval(x)] if x != "[]" else {})

In [6]:
movies['geners_name']

0       [Action, Adventure, Fantasy, Science Fiction]
1                        [Adventure, Fantasy, Action]
2                          [Action, Adventure, Crime]
3                    [Action, Crime, Drama, Thriller]
4                [Action, Adventure, Science Fiction]
                            ...                      
4798                        [Action, Crime, Thriller]
4799                                [Comedy, Romance]
4800               [Comedy, Drama, Romance, TV Movie]
4801                                               {}
4802                                    [Documentary]
Name: geners_name, Length: 4803, dtype: object

##create a dataset to train and test

In [7]:
movies = movies[['title','id','overview','geners_name','keywords']]
movies.dropna(inplace=True)

In [28]:
movies['tags'] = movies['overview'] + " "+ movies['geners_name'].apply(lambda x: " ".join(x))*4 + " "+ movies['keywords'] +" "+ movies['title']

In [29]:
movies['tags']

0       In the 22nd century, a paraplegic Marine is di...
1       Captain Barbossa, long believed to be dead, ha...
2       A cryptic message from Bond’s past sends him o...
3       Following the death of District Attorney Harve...
4       John Carter is a war-weary, former military ca...
                              ...                        
4798    El Mariachi just wants to play his guitar and ...
4799    A newlywed couple's honeymoon is upended by th...
4800    "Signed, Sealed, Delivered" introduces a dedic...
4801    When ambitious New York attorney Sam is sent t...
4802    Ever since the second grade when he first saw ...
Name: tags, Length: 4800, dtype: object

In [30]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
movies['tags'] = movies['tags'].apply(lambda x: " ".join([ps.stem(word) for word in x.split()]))

In [31]:
#model training

In [32]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer

word vectorize


In [33]:
vectorizer = TfidfVectorizer(max_features=5000,stop_words='english')
vectors = vectorizer.fit_transform(movies['tags']).toarray() # type: ignore

In [34]:
#model
similarty = cosine_similarity(vectors)

In [ ]:
from typing import Any


def recommend(movie:str) -> list[Any]:
    # movie = movie.capitalize()
    movie_index = movies[movies['title'] == movie].index[0]
    distance = similarty[movie_index]
    movie_list = sorted(list(enumerate(distance)),reverse=True,key=lambda x: x[1])[1:6]
    
    target_gener = movies.iloc[movie_index].genres.split()[0]
    recommendations = [
        movies.iloc[i[0]].title for i in movie_list
        if target_gener in movies.iloc[i[0]].geners
    ]

In [39]:
recommend("Avatar")

['Avatar', 'Avatar', 'Avatar', 'Avatar', 'Avatar']